In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, RobertaTokenizer, RobertaForSequenceClassification
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_excel('/content/dataset_final.xlsx')

# Print out the column names to inspect them
print("Columns in the dataset:", df.columns)

# Replace 'Question' and 'Discipline' with your actual column names
df = df[['Question', 'Discipline']]  # Assuming 'Question' is text and 'Discipline' is the label column
# Remove rows with missing values
df = df.dropna()

# Label Encoding for the 'Discipline' column
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Discipline'])

# Train-Test Split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert the data to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Choose a model ('bert-base-uncased' or 'roberta-base')
model_name = 'bert-base-uncased'

if model_name.startswith('bert'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())
elif model_name.startswith('roberta'):
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())

# Tokenization Function
def tokenize_function(examples):
    return tokenizer(examples['Question'], padding='max_length', truncation=True)

# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Ensure 'Discipline' is encoded as 'label'
train_dataset = train_dataset.rename_column('label', 'labels')
test_dataset = test_dataset.rename_column('label', 'labels')

# Set format for PyTorch (ensure 'labels' is included)
target_columns = ['input_ids', 'attention_mask', 'labels']
train_dataset.set_format(type='torch', columns=target_columns)
test_dataset.set_format(type='torch', columns=target_columns)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)  # Print out the evaluation results for comparison purposes



Columns in the dataset: Index(['Discipline', 'Question', 'Reason you think it is faulty',
       'Which top LLM you tried ', 'Response by a top LLM '],
      dtype='object')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3523 [00:00<?, ? examples/s]

Map:   0%|          | 0/881 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [3]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_excel('/content/dataset_final.xlsx')

# Print out the column names to inspect them
print("Columns in the dataset:", df.columns)

# Replace 'Question' and 'Discipline' with your actual column names
df = df[['Question', 'Discipline']]  # Assuming 'Question' is text and 'Discipline' is the label column
# Remove rows with missing values
df = df.dropna()

# Label Encoding for the 'Discipline' column
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Discipline'])

# Train-Test Split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert the data to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Define function to load models and tokenizers
def load_model_and_tokenizer(model_name):
    if model_name.startswith('bert'):
        tokenizer = BertTokenizer.from_pretrained(model_name)
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())
    elif model_name.startswith('roberta'):
        tokenizer = RobertaTokenizer.from_pretrained(model_name)
        model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())
    return tokenizer, model

# Tokenization Function
def tokenize_function(examples):
    return tokenizer(examples['Question'], padding='max_length', truncation=True)

# Define function for metrics (Accuracy, Precision, Recall, F1)
def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define function to train and evaluate the model
def train_and_evaluate(model_name, batch_size=8, learning_rate=2e-5, epochs=3):
    # Load model and tokenizer
    tokenizer, model = load_model_and_tokenizer(model_name)

    # Tokenize the datasets
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Ensure 'Discipline' is encoded as 'label'
    train_dataset = train_dataset.rename_column('label', 'labels')
    test_dataset = test_dataset.rename_column('label', 'labels')

    # Set format for PyTorch (ensure 'labels' is included)
    target_columns = ['input_ids', 'attention_mask', 'labels']
    train_dataset.set_format(type='torch', columns=target_columns)
    test_dataset.set_format(type='torch', columns=target_columns)

    # Define Training Arguments
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy='epoch',
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01,
        report_to="none"  # Avoid logging to Weights & Biases for now
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    print(f"Evaluation Results for {model_name}:")
    print(eval_results)  # Print out the evaluation results for comparison

# Experiment with different hyperparameters and models
print("Training BERT model...")
train_and_evaluate('bert-base-uncased', batch_size=8, learning_rate=2e-5, epochs=3)

print("\nTraining RoBERTa model...")
train_and_evaluate('roberta-base', batch_size=8, learning_rate=2e-5, epochs=3)

# Hyperparameter Tuning Example:
# You can experiment with different learning rates, batch sizes, and epochs

print("\nTraining BERT with a different learning rate (1e-5)...")
train_and_evaluate('bert-base-uncased', batch_size=8, learning_rate=1e-5, epochs=3)

print("\nTraining RoBERTa with a different batch size (16)...")
train_and_evaluate('roberta-base', batch_size=16, learning_rate=2e-5, epochs=3)


Columns in the dataset: Index(['Discipline', 'Question', 'Reason you think it is faulty',
       'Which top LLM you tried ', 'Response by a top LLM '],
      dtype='object')
Training BERT model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


UnboundLocalError: local variable 'train_dataset' referenced before assignment

In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_excel('/content/dataset_final.xlsx')

# Print out the column names to inspect them
print("Columns in the dataset:", df.columns)

# Replace 'Question' and 'Discipline' with your actual column names
df = df[['Question', 'Discipline']]  # Assuming 'Question' is text and 'Discipline' is the label column
# Remove rows with missing values
df = df.dropna()

# Label Encoding for the 'Discipline' column
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Discipline'])

# Train-Test Split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert the data to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Define function to load models and tokenizers
def load_model_and_tokenizer(model_name):
    if model_name.startswith('bert'):
        tokenizer = BertTokenizer.from_pretrained(model_name)
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())
    elif model_name.startswith('roberta'):
        tokenizer = RobertaTokenizer.from_pretrained(model_name)
        model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())
    return tokenizer, model

# Tokenization Function
def tokenize_function(examples):
    return tokenizer(examples['Question'], padding='max_length', truncation=True)

# Define function for metrics (Accuracy, Precision, Recall, F1)
def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define function to train and evaluate the model
def train_and_evaluate(model_name, batch_size=8, learning_rate=2e-5, epochs=3):
    # Load model and tokenizer
    tokenizer, model = load_model_and_tokenizer(model_name)

    # Tokenize the datasets
    train_tokenized = train_dataset.map(tokenize_function, batched=True)
    test_tokenized = test_dataset.map(tokenize_function, batched=True)

    # Ensure 'Discipline' is encoded as 'labels'
    train_tokenized = train_tokenized.rename_column('label', 'labels')
    test_tokenized = test_tokenized.rename_column('label', 'labels')

    # Set format for PyTorch (ensure 'labels' is included)
    target_columns = ['input_ids', 'attention_mask', 'labels']
    train_tokenized.set_format(type='torch', columns=target_columns)
    test_tokenized.set_format(type='torch', columns=target_columns)

    # Define Training Arguments
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy='epoch',
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01,
        report_to="none"  # Avoid logging to Weights & Biases for now
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=test_tokenized,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    print(f"Evaluation Results for {model_name}:")
    print(eval_results)  # Print out the evaluation results for comparison

# Experiment with different hyperparameters and models
print("Training BERT model...")
train_and_evaluate('bert-base-uncased', batch_size=8, learning_rate=2e-5, epochs=3)

print("\nTraining RoBERTa model...")
train_and_evaluate('roberta-base', batch_size=8, learning_rate=2e-5, epochs=3)

# Hyperparameter Tuning Example:
# You can experiment with different learning rates, batch sizes, and epochs

print("\nTraining BERT with a different learning rate (1e-5)...")
train_and_evaluate('bert-base-uncased', batch_size=8, learning_rate=1e-5, epochs=3)

print("\nTraining RoBERTa with a different batch size (16)...")
train_and_evaluate('roberta-base', batch_size=16, learning_rate=2e-5, epochs=3)


Columns in the dataset: Index(['Discipline', 'Question', 'Reason you think it is faulty',
       'Which top LLM you tried ', 'Response by a top LLM '],
      dtype='object')
Training BERT model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3523 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

In [5]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_excel('/content/dataset_final.xlsx')

# Print out the column names to inspect them
print("Columns in the dataset:", df.columns)

# Replace 'Question' and 'Discipline' with your actual column names
df = df[['Question', 'Discipline']]  # Assuming 'Question' is text and 'Discipline' is the label column
# Remove rows with missing values
df = df.dropna()

# Label Encoding for the 'Discipline' column
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Discipline'])

# Train-Test Split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert the data to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Define function to load models and tokenizers
def load_model_and_tokenizer(model_name):
    if model_name.startswith('bert'):
        tokenizer = BertTokenizer.from_pretrained(model_name)
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())
    elif model_name.startswith('roberta'):
        tokenizer = RobertaTokenizer.from_pretrained(model_name)
        model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())
    return tokenizer, model

# Tokenization Function
def tokenize_function(examples, tokenizer):
    return tokenizer(examples['Question'], padding='max_length', truncation=True)

# Define function for metrics (Accuracy, Precision, Recall, F1)
def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define function to train and evaluate the model
def train_and_evaluate(model_name, batch_size=8, learning_rate=2e-5, epochs=3):
    # Load model and tokenizer
    tokenizer, model = load_model_and_tokenizer(model_name)

    # Tokenize the datasets
    train_tokenized = train_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    test_tokenized = test_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

    # Ensure 'Discipline' is encoded as 'labels'
    train_tokenized = train_tokenized.rename_column('label', 'labels')
    test_tokenized = test_tokenized.rename_column('label', 'labels')

    # Set format for PyTorch (ensure 'labels' is included)
    target_columns = ['input_ids', 'attention_mask', 'labels']
    train_tokenized.set_format(type='torch', columns=target_columns)
    test_tokenized.set_format(type='torch', columns=target_columns)

    # Define Training Arguments
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy='epoch',
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01,
        report_to="none"  # Avoid logging to Weights & Biases for now
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=test_tokenized,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    print(f"Evaluation Results for {model_name}:")
    print(eval_results)  # Print out the evaluation results for comparison

# Experiment with different hyperparameters and models
print("Training BERT model...")
train_and_evaluate('bert-base-uncased', batch_size=8, learning_rate=2e-5, epochs=3)

print("\nTraining RoBERTa model...")
train_and_evaluate('roberta-base', batch_size=8, learning_rate=2e-5, epochs=3)

# Hyperparameter Tuning Example:
# You can experiment with different learning rates, batch sizes, and epochs

print("\nTraining BERT with a different learning rate (1e-5)...")
train_and_evaluate('bert-base-uncased', batch_size=8, learning_rate=1e-5, epochs=3)

print("\nTraining RoBERTa with a different batch size (16)...")
train_and_evaluate('roberta-base', batch_size=16, learning_rate=2e-5, epochs=3)


Columns in the dataset: Index(['Discipline', 'Question', 'Reason you think it is faulty',
       'Which top LLM you tried ', 'Response by a top LLM '],
      dtype='object')
Training BERT model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3523 [00:00<?, ? examples/s]

Map:   0%|          | 0/881 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Map:   0%|          | 0/3523 [00:00<?, ? examples/s]

Map:   0%|          | 0/881 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.115379,0.969353,0.970820,0.969353,0.969288
2,No log,0.045972,0.990919,0.991284,0.990919,0.990917
3,0.367200,0.015079,0.996595,0.996635,0.996595,0.996595


Evaluation Results for roberta-base:
{'eval_loss': 0.015079336240887642, 'eval_accuracy': 0.996594778660613, 'eval_precision': 0.9966354995355018, 'eval_recall': 0.996594778660613, 'eval_f1': 0.9965951798214873, 'eval_runtime': 22.9924, 'eval_samples_per_second': 38.317, 'eval_steps_per_second': 2.436, 'epoch': 3.0}
